In [4]:
import json
import pandas as pd
import numpy as np

In [5]:
from scipy.spatial.distance import cdist

In [6]:
with open('offices.json', encoding='utf-8') as f:
    offices = json.load(f)

In [7]:
kremlin_lat = 55.752
kremlin_lon = 37.618
degree_to_meter = 111319
moscow_rad = 70000 / degree_to_meter
office_latitudes = []
office_longitudes = []
max_people = []
opening_times = [[] for _ in range(7)]
closing_times = [[] for _ in range(7)]
business_morning = [[] for _ in range(7)]
business_day = [[] for _ in range(7)]
business_evening = [[] for _ in range(7)]

In [8]:
ord_to_day = {0: 'mn', 1: 'ts', 2: 'wd', 3: 'th', 4: 'fr', 5: 'st', 6: 'sn'}
day_to_ord = {'пн': 0, 'вт': 1, 'ср': 2, 'чт': 3, 'пт': 4, 'сб': 5, 'вс': 6}
ans = []
for obj in offices:
    if  'Не' in obj['openHours'][0]['days']:
        continue

    if len(obj['openHours']) == 7:
        ans.append((obj['openHours'] + [float(obj['latitude']), float(obj['longitude'])]).copy())
        continue

    new_dict_list = list()
    for l in obj['openHours']:
        if l["days"] == 'перерыв':
            for i in range(len(new_dict_list)):
                new_dict_list[i]['hours'] = new_dict_list[i]['hours'][:5] + '-' + l['hours'][:5] + ', ' + l['hours'][6:] + '-' + new_dict_list[i]['hours'][6:]
        elif len(l["days"]) > 2:
            s_front = l["days"][:2]
            s_back = l["days"][3:]

            for i in range(day_to_ord[s_front], day_to_ord[s_back] + 1):
                if i + 1 <= len(new_dict_list):
                    continue
                new_dict_list.append({'days': ord_to_day[i], "hours": l["hours"]})
        else:
            new_dict_list.append({"days": l["days"], "hours": l["hours"]})
    ans.append((new_dict_list + [float(obj['latitude']), float(obj['longitude'])]).copy())

In [9]:
ans

[[{'days': 'пн', 'hours': '09:00-18:00'},
  {'days': 'вт', 'hours': '09:00-18:00'},
  {'days': 'ср', 'hours': '09:00-18:00'},
  {'days': 'чт', 'hours': '09:00-18:00'},
  {'days': 'пт', 'hours': '09:00-17:00'},
  {'days': 'сб', 'hours': 'выходной'},
  {'days': 'вс', 'hours': 'выходной'},
  56.184479,
  36.984314],
 [{'days': 'mn', 'hours': '09:00-18:00'},
  {'days': 'ts', 'hours': '09:00-18:00'},
  {'days': 'wd', 'hours': '09:00-18:00'},
  {'days': 'th', 'hours': '09:00-18:00'},
  {'days': 'пт', 'hours': '09:00-17:00'},
  {'days': 'st', 'hours': 'выходной'},
  {'days': 'sn', 'hours': 'выходной'},
  56.183239,
  36.9757],
 [{'days': 'пн', 'hours': '10:00-19:00'},
  {'days': 'вт', 'hours': '10:00-19:00'},
  {'days': 'ср', 'hours': '10:00-19:00'},
  {'days': 'чт', 'hours': '10:00-19:00'},
  {'days': 'пт', 'hours': '10:00-18:00'},
  {'days': 'сб', 'hours': 'выходной'},
  {'days': 'вс', 'hours': 'выходной'},
  56.012386,
  37.482059],
 [{'days': 'пн', 'hours': '09:00-18:00'},
  {'days': 'вт'

In [10]:
for elem in ans:
    office_latitudes.append(elem[7])
    office_longitudes.append(elem[8])
    max_people.append(np.random.randint(1000, 3000))
    for i in range(7):
        opening_times[i].append(elem[i]['hours'][:5] if elem[i]['hours'] != 'выходной' else None)
        closing_times[i].append(elem[i]['hours'][-5:] if elem[i]['hours'] != 'выходной' else None)
        business_morning[i].append(round(np.random.uniform(0.01, 0.5), 2))
        business_day[i].append(round(np.random.uniform(0.01, 1), 2))
        business_evening[i].append(round(np.random.uniform(0.01, 0.5), 2))

In [11]:
len(office_latitudes)

160

In [12]:
len(office_longitudes)

160

In [13]:
len(max_people)

160

In [14]:
for i in range(7):
    print(len(opening_times[i]))
    print(len(closing_times[i]))
    print(len(business_morning[i]))
    print(len(business_day[i]))
    print(len(business_evening[i]))

160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160
160


In [15]:
db = pd.DataFrame()
db['Latitude'] = pd.Series(office_latitudes)

In [16]:
db

,Latitude
0,56.184479
1,56.183239
2,56.012386
3,56.010849
4,56.008335
...,...
155,55.478329
156,55.438740
157,55.432919
158,55.427883


In [17]:
db['Longitude'] = pd.Series(office_longitudes)
db['Max_People'] = pd.Series(max_people)
for i in range(7):
    db[f'opening_{ord_to_day[i]}'] = pd.Series(opening_times[i]).str[:2].fillna(0).astype(int)
    db[f'closing_{ord_to_day[i]}'] = pd.Series(closing_times[i]).str[:2].fillna(0).astype(int)
    db[f'morning_{ord_to_day[i]}'] = pd.Series(business_morning[i])
    db[f'day_{ord_to_day[i]}'] = pd.Series(business_day[i])
    db[f'evening_{ord_to_day[i]}'] = pd.Series(business_evening[i])

In [18]:
db

,Latitude,Longitude,Max_People,opening_mn,closing_mn,morning_mn,day_mn,evening_mn,opening_ts,closing_ts,...,opening_st,closing_st,morning_st,day_st,evening_st,opening_sn,closing_sn,morning_sn,day_sn,evening_sn
0,56.184479,36.984314,1119,9,18,0.30,0.58,0.32,9,18,...,0,0,0.35,0.08,0.35,0,0,0.04,0.61,0.37
1,56.183239,36.975700,1527,9,18,0.26,0.56,0.06,9,18,...,0,0,0.45,0.95,0.27,0,0,0.39,0.38,0.43
2,56.012386,37.482059,2388,10,19,0.34,0.01,0.06,10,19,...,0,0,0.05,0.24,0.17,0,0,0.17,0.24,0.37
3,56.010849,37.854359,1402,9,18,0.08,0.97,0.14,9,18,...,0,0,0.46,0.56,0.01,0,0,0.22,0.41,0.26
4,56.008335,37.851467,2963,9,18,0.48,0.32,0.05,9,18,...,0,0,0.46,0.19,0.05,0,0,0.15,0.08,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,55.478329,37.298706,1258,9,18,0.15,0.73,0.15,9,18,...,0,0,0.03,0.34,0.25,0,0,0.49,0.11,0.07
156,55.438740,37.767536,1596,9,18,0.45,0.87,0.26,9,18,...,0,0,0.44,0.22,0.44,0,0,0.38,0.19,0.35
157,55.432919,37.550838,2805,9,18,0.06,0.35,0.32,9,18,...,0,0,0.11,0.45,0.22,0,0,0.48,0.66,0.09
158,55.427883,37.550110,2825,9,18,0.03,0.39,0.06,9,18,...,0,0,0.17,0.79,0.30,0,0,0.35,0.22,0.46


In [19]:
examples_quantity = 8192
examples = pd.DataFrame()
usr_lat = []
usr_lon = []
days = [np.random.randint(0, 5) for i in range(examples_quantity)]
time = [np.random.uniform(9, 19) for i in range(examples_quantity)]
for i in range(examples_quantity):
    rho = np.random.uniform(0, moscow_rad)
    phi = np.random.uniform(0, 2 * np.pi)
    usr_lat.append(kremlin_lat + rho * np.sin(phi))
    usr_lon.append(kremlin_lon + rho * np.cos(phi))

In [20]:
examples['Latitude'] = usr_lat
examples['Longitude'] = usr_lon
examples['day'] = days
examples['time'] = time

In [21]:
examples

,Latitude,Longitude,day,time
0,55.732761,37.585269,0,10.150501
1,55.135852,37.620822,4,11.112689
2,55.694201,37.713336,3,11.992098
3,55.818635,37.763285,0,17.870403
4,55.615080,37.563649,2,11.058764
...,...,...,...,...
8187,55.820093,37.702651,2,15.053596
8188,55.211948,37.368396,2,17.873015
8189,55.763741,37.589529,4,16.355091
8190,55.415649,37.842683,2,13.254130


In [22]:
db

,Latitude,Longitude,Max_People,opening_mn,closing_mn,morning_mn,day_mn,evening_mn,opening_ts,closing_ts,...,opening_st,closing_st,morning_st,day_st,evening_st,opening_sn,closing_sn,morning_sn,day_sn,evening_sn
0,56.184479,36.984314,1119,9,18,0.30,0.58,0.32,9,18,...,0,0,0.35,0.08,0.35,0,0,0.04,0.61,0.37
1,56.183239,36.975700,1527,9,18,0.26,0.56,0.06,9,18,...,0,0,0.45,0.95,0.27,0,0,0.39,0.38,0.43
2,56.012386,37.482059,2388,10,19,0.34,0.01,0.06,10,19,...,0,0,0.05,0.24,0.17,0,0,0.17,0.24,0.37
3,56.010849,37.854359,1402,9,18,0.08,0.97,0.14,9,18,...,0,0,0.46,0.56,0.01,0,0,0.22,0.41,0.26
4,56.008335,37.851467,2963,9,18,0.48,0.32,0.05,9,18,...,0,0,0.46,0.19,0.05,0,0,0.15,0.08,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,55.478329,37.298706,1258,9,18,0.15,0.73,0.15,9,18,...,0,0,0.03,0.34,0.25,0,0,0.49,0.11,0.07
156,55.438740,37.767536,1596,9,18,0.45,0.87,0.26,9,18,...,0,0,0.44,0.22,0.44,0,0,0.38,0.19,0.35
157,55.432919,37.550838,2805,9,18,0.06,0.35,0.32,9,18,...,0,0,0.11,0.45,0.22,0,0,0.48,0.66,0.09
158,55.427883,37.550110,2825,9,18,0.03,0.39,0.06,9,18,...,0,0,0.17,0.79,0.30,0,0,0.35,0.22,0.46


In [23]:
points1 = db.loc[:, ['Latitude', 'Longitude']].values

In [24]:
points2 = examples.loc[:, ['Latitude', 'Longitude']].values

In [25]:
distances = cdist(points1, points2)
distances_df = pd.DataFrame(distances, index=db.index, columns=['dist' + str(idx) for idx in examples.index])
distances_df

,dist0,dist1,dist2,dist3,dist4,dist5,dist6,dist7,dist8,dist9,...,dist8182,dist8183,dist8184,dist8185,dist8186,dist8187,dist8188,dist8189,dist8190,dist8191
0,0.751795,1.226687,0.878548,0.860603,0.812308,0.640636,0.511909,1.001189,0.820446,0.358940,...,0.955861,1.099234,0.659578,0.745020,0.572579,0.805473,1.045627,0.737093,1.152344,0.781167
1,0.757960,1.230123,0.885023,0.867886,0.817612,0.644616,0.515404,1.007780,0.826232,0.362373,...,0.964221,1.107220,0.668066,0.748869,0.576880,0.812609,1.047672,0.743481,1.157952,0.788018
2,0.298064,0.887450,0.393359,0.341508,0.405597,0.375399,0.351237,0.505023,0.381201,0.342322,...,0.450431,0.574747,0.198927,0.451563,0.319804,0.292639,0.808468,0.270877,0.697241,0.282712
3,0.386965,0.905627,0.346632,0.212699,0.491066,0.612400,0.658062,0.368915,0.433574,0.699430,...,0.086411,0.223113,0.244254,0.632771,0.592879,0.243728,0.935095,0.362212,0.595314,0.291859
4,0.383148,0.902454,0.343163,0.209194,0.487328,0.608590,0.654407,0.366199,0.429867,0.696110,...,0.090145,0.224292,0.243000,0.628946,0.589142,0.239961,0.931445,0.358383,0.592751,0.288059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.383215,0.470159,0.467460,0.575883,0.298154,0.189529,0.273484,0.552823,0.356490,0.418703,...,0.853017,0.853885,0.737357,0.137084,0.253709,0.529126,0.275346,0.407477,0.547576,0.479235
156,0.345933,0.336551,0.261147,0.379918,0.269566,0.470516,0.601675,0.212575,0.266358,0.742097,...,0.633755,0.533011,0.703641,0.387218,0.523567,0.386833,0.459072,0.370556,0.078614,0.370052
157,0.301813,0.305199,0.307691,0.440352,0.182611,0.301684,0.434548,0.346533,0.227832,0.585635,...,0.725627,0.675599,0.704014,0.200559,0.368626,0.415873,0.286554,0.333077,0.292355,0.376239
158,0.306899,0.300470,0.312359,0.445118,0.187686,0.305022,0.437712,0.350183,0.232886,0.588982,...,0.730302,0.679576,0.709102,0.203454,0.372268,0.420829,0.282220,0.338163,0.292828,0.381269


In [26]:
examples['day'].values

array([0, 4, 3, ..., 4, 2, 2], dtype=int64)

In [27]:
diff_matrix = [[0] * 8192 for _ in range(160)]
days = examples['day'].values
coming = examples['time'].values
for i in range(160):
    for j in range(8192):
        closing_hours = db[f'closing_{ord_to_day[days[j]]}'].values
        diff_matrix[i][j] = 1 / max(closing_hours[i] - coming[j], 0.0001)

In [28]:
diff_df = pd.DataFrame(diff_matrix, index=db.index, columns=['diff' + str(idx) for idx in examples.index])

In [29]:
diff_df

,diff0,diff1,diff2,diff3,diff4,diff5,diff6,diff7,diff8,diff9,...,diff8182,diff8183,diff8184,diff8185,diff8186,diff8187,diff8188,diff8189,diff8190,diff8191
0,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
1,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
2,0.113001,0.145195,0.142696,0.885272,0.125925,4.242311,0.197919,0.611074,0.234431,0.104373,...,3.185313,7.767311,0.101217,0.407274,0.191073,0.253395,0.887323,0.607937,0.174038,0.152482
3,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
4,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
156,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
157,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915
158,0.127397,0.169857,0.166447,7.716238,0.144067,10000.000000,0.246758,1.571186,0.306219,0.116536,...,10000.000000,10000.000000,0.112616,0.687121,0.236206,0.339397,7.874932,1.550608,0.210710,0.179915


In [30]:
busy_matrix = [[0] * 8192 for _ in range(160)]
days = examples['day'].values
time = examples['time'].values
for i in range(160):
    for j in range(8192):
        if time[j] < 12:
            business = db[f'morning_{ord_to_day[days[j]]}'].values
        elif time[j] > 15:
            business = db[f'evening_{ord_to_day[days[j]]}'].values
        else:
            business = db[f'day_{ord_to_day[days[j]]}'].values
        busy_matrix[i][j] = business[i]


In [31]:
busy_df = pd.DataFrame(busy_matrix, index=db.index, columns=['busy' + str(idx) for idx in examples.index])

In [32]:
busy_df

,busy0,busy1,busy2,busy3,busy4,busy5,busy6,busy7,busy8,busy9,...,busy8182,busy8183,busy8184,busy8185,busy8186,busy8187,busy8188,busy8189,busy8190,busy8191
0,0.30,0.23,0.48,0.32,0.24,0.32,0.83,0.32,0.77,0.24,...,0.38,0.32,0.15,0.32,0.52,0.42,0.42,0.21,0.83,0.83
1,0.26,0.23,0.24,0.06,0.19,0.06,1.00,0.06,0.18,0.19,...,0.33,0.06,0.31,0.06,0.17,0.03,0.03,0.31,1.00,1.00
2,0.34,0.44,0.04,0.06,0.09,0.06,0.84,0.06,0.87,0.09,...,0.38,0.06,0.43,0.06,0.18,0.49,0.49,0.01,0.84,0.84
3,0.08,0.03,0.04,0.14,0.04,0.14,0.37,0.14,0.30,0.04,...,0.16,0.14,0.30,0.14,0.92,0.43,0.43,0.43,0.37,0.37
4,0.48,0.42,0.21,0.05,0.18,0.05,0.21,0.05,0.70,0.18,...,0.43,0.05,0.28,0.05,0.41,0.48,0.48,0.48,0.21,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.15,0.25,0.20,0.15,0.05,0.15,0.99,0.15,0.23,0.05,...,0.33,0.15,0.20,0.15,0.94,0.26,0.26,0.31,0.99,0.99
156,0.45,0.41,0.35,0.26,0.03,0.26,0.91,0.26,0.13,0.03,...,0.26,0.26,0.30,0.26,0.59,0.23,0.23,0.20,0.91,0.91
157,0.06,0.29,0.26,0.32,0.27,0.32,0.33,0.32,0.82,0.27,...,0.22,0.32,0.33,0.32,0.87,0.49,0.49,0.27,0.33,0.33
158,0.03,0.34,0.17,0.06,0.26,0.06,0.81,0.06,0.75,0.26,...,0.18,0.06,0.04,0.06,0.19,0.38,0.38,0.30,0.81,0.81


In [33]:
vect = []
for i in range(8192):
    dist = distances_df.loc[:, f'dist{i}']
    diff = diff_df.loc[:, f'diff{i}']
    busy = busy_df.loc[:, f'busy{i}']
    result = pd.concat([dist, diff, busy], axis=1, join="outer")
    result = result.sort_values(by=[f'dist{i}', f'diff{i}', f'busy{i}'])
    vect.append(list(result.loc[[result.index[0], result.index[-1]]].values))

In [34]:
vect

[[array([0.00476898, 0.12739666, 0.21      ]),
  array([0.75796046, 0.12739666, 0.26      ])],
 [array([0.30047019, 0.16985683, 0.34      ]),
  array([1.23012271, 0.16985683, 0.23      ])],
 [array([0.04709827, 0.16644745, 0.49      ]),
  array([0.88502287, 0.16644745, 0.24      ])],
 [array([0.04359933, 0.8852716 , 0.37      ]),
  array([0.91378526, 7.7162377 , 0.06      ])],
 [array([0.02892396, 0.14406655, 0.17      ]),
  array([0.81761178, 0.14406655, 0.19      ])],
 [array([4.31524591e-02, 1.00000000e+04, 2.30000000e-01]),
  array([9.39099721e-01, 1.00000000e+04, 2.80000000e-01])],
 [array([0.01199263, 0.2467575 , 0.54      ]),
  array([1.06081768, 0.2467575 , 0.24      ])],
 [array([0.02717356, 0.61107449, 0.12      ]),
  array([1.00777975, 1.57118644, 0.06      ])],
 [array([0.01117123, 0.23443143, 0.53      ]),
  array([0.82623181, 0.30621871, 0.18      ])],
 [array([0.04948284, 0.11653615, 0.18      ]),
  array([1.18346345, 0.11653615, 0.2       ])],
 [array([0.18443553, 0.117

In [35]:
X = np.reshape(np.array(vect), (-1, 3))

In [36]:
y = np.reshape(np.array([[0, 1] for _ in range(8192)]), (8192*2))

In [37]:
y = np.reshape(y, 8192*2)

In [38]:
from skopt import BayesSearchCV
from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [39]:
from torch.autograd import Variable
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch
import matplotlib.pyplot as plt

import os
import pandas as pd
import skimage.io
from skimage.transform import resize


In [40]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [41]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# работаем на видеокарте
DEVICE = torch.device("cuda")

EPOCHS = 1000
BATCH_SIZE = 256

In [42]:
class JustSigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
                nn.Linear(3, 1)
            )
    def forward(self, x):
        return self.layer(x)

In [43]:
lean = JustSigmoid().to(DEVICE)
print(lean)

JustSigmoid(
  (layer): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
  )
)


In [45]:
lean.load_state_dict(torch.load('Weights/lean'))

<All keys matched successfully>

In [49]:
from torch.utils.data import DataLoader, Dataset
criterion = F.mse_loss

optimizer = torch.optim.Adam(lean.parameters(), lr=0.001)
train_losses = []
val_losses = []
X = torch.Tensor(X)
y = torch.Tensor(y)
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.9)
# train_dataset = Dataset(train_X, train_y)
# val_dataset = Dataset(val_X, val_y)
train_loader = DataLoader(list(zip(train_X, train_y)), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(list(zip(val_X, val_y)), batch_size=BATCH_SIZE, shuffle=True)

In [103]:
from tqdm import notebook

for epoch in notebook.tqdm(range(EPOCHS)):
    lean.train()
    train_losses_per_epoch = []
    for X_batch, y_batch in train_loader:
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        calculated = torch.reshape(lean(torch.Tensor(X_batch).to(DEVICE)), (-1,))
        loss = criterion(calculated, torch.Tensor(y_batch).to(DEVICE))
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    lean.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            calculated = torch.reshape(lean(torch.Tensor(X_batch).to(DEVICE)), (-1,))
            loss = criterion(calculated, torch.Tensor(y_batch).to(DEVICE))
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [104]:
with torch.no_grad():
    calc = torch.reshape(lean(torch.Tensor([[0.03095769, 0.2211782 , 0.93], [0.9236934 , 0.19648128, 0.55]]).to(DEVICE)), (-1,))

In [105]:
calc

tensor([-0.0135,  0.8823], device='cuda:0')

In [107]:
torch.save(lean.state_dict(), 'weights/lean')

In [109]:
java = torch.jit.trace(lean, torch.Tensor(val_X).to(DEVICE))

In [110]:
java.save('./model1.zip')

In [46]:
lean

JustSigmoid(
  (layer): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [51]:
import torch.onnx
import onnx

In [2]:
!pip install onnx

In [50]:
torch.onnx.export(lean, torch.Tensor(val_X).to(DEVICE), "Models/model.onnx")

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

